This is the second assignment for the Coursera course "Advanced Machine Learning and Signal Processing"


Just execute all cells one after the other and you are done - just note that in the last one you have to update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"

In [0]:
credentials_1 = {'password':"""*****""",
                 'custom_url':'*******',
                 'username':'*****' }




Let's create a SparkSession object and put the Cloudant credentials into it

In [0]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",credentials_1['custom_url'].split('@')[1])\
    .config("cloudant.username", credentials_1['username'])\
    .config("cloudant.password",credentials_1['password'])\
    .getOrCreate()

Now it’s time to have a look at the recorded sensor data. You should see data similar to the one exemplified below….


In [0]:
df=spark.read.load('shake_classification', "org.apache.bahir.cloudant")

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()


+-----+--------+-----+-----+-----+--------------------+--------------------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|
+-----+--------+-----+-----+-----+--------------------+--------------------+
|    1|aaaaaaaa|-0.01|-0.03|-0.09|3492765e6ac38cd24...|1-2c827a2b55caa1e...|
|    1|aaaaaaaa|-0.01|  0.0| -0.1|3492765e6ac38cd24...|1-86ceff170907fc5...|
|    1|aaaaaaaa| 0.01|  0.0|-0.07|3492765e6ac38cd24...|1-22ba2f3ccff5cde...|
|    1|aaaaaaaa|-0.01|  0.0|-0.08|3492765e6ac38cd24...|1-978a5470abb20cb...|
|    1|aaaaaaaa| 0.01|  0.0|-0.06|3492765e6ac38cd24...|1-96f8550e65d9c9f...|
|    1|aaaaaaaa|  0.0|  0.0|-0.09|3492765e6ac38cd24...|1-1f5ca0558b17a16...|
|    1|aaaaaaaa| 0.01|  0.0|-0.07|3492765e6ac38cd24...|1-22ba2f3ccff5cde...|
|    1|aaaaaaaa|  0.0|  0.0|-0.07|3492765e6ac38cd24...|1-8fe62ca7a975b42...|
|    1|aaaaaaaa|  0.0|  0.0|-0.07|3492765e6ac38cd24...|1-8fe62ca7a975b42...|
|    1|aaaaaaaa|-0.01|  0.0|-0.08|3492765e6ac38cd24...|1-978a5470abb20cb...|

Please create a VectorAssembler which consumed columns X, Y and Z and produces a column “features”


In [0]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols=["X", "Y", "Z"], outputCol="features")

Please insatiate a classifier from the SparkML package and assign it to the classifier variable. Make sure to either
1.	Rename the “CLASS” column to “label” or
2.	Specify the label-column correctly to be “CLASS”


In [0]:
from pyspark.ml.classification import GBTClassifier

classifier = GBTClassifier(labelCol='CLASS', featuresCol='features', maxIter=10)



Let’s train and evaluate…


In [0]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [0]:
model = pipeline.fit(df)

In [0]:
prediction = model.transform(df)

In [0]:
prediction.show()

+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|           features|prediction|
+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|    1|aaaaaaaa|-0.01|-0.03|-0.09|3492765e6ac38cd24...|1-2c827a2b55caa1e...|[-0.01,-0.03,-0.09]|       1.0|
|    1|aaaaaaaa|-0.01|  0.0| -0.1|3492765e6ac38cd24...|1-86ceff170907fc5...|   [-0.01,0.0,-0.1]|       1.0|
|    1|aaaaaaaa| 0.01|  0.0|-0.07|3492765e6ac38cd24...|1-22ba2f3ccff5cde...|   [0.01,0.0,-0.07]|       1.0|
|    1|aaaaaaaa|-0.01|  0.0|-0.08|3492765e6ac38cd24...|1-978a5470abb20cb...|  [-0.01,0.0,-0.08]|       1.0|
|    1|aaaaaaaa| 0.01|  0.0|-0.06|3492765e6ac38cd24...|1-96f8550e65d9c9f...|   [0.01,0.0,-0.06]|       1.0|
|    1|aaaaaaaa|  0.0|  0.0|-0.09|3492765e6ac38cd24...|1-1f5ca0558b17a16...|    [0.0,0.0,-0.09]|       1.0|
|    1|aaaaaaaa| 0.01|  0.0|

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("CLASS")
    
binEval.evaluate(prediction) 

0.8612754766600921

If you are happy with the result (I’m happy with > 0.55) please submit your solution to the grader by executing the following cells, please don’t forget to obtain an assignment submission token (secret) from the Courera’s graders web page and paste it to the “secret” variable below, including your email address you’ve used for Coursera. (0.55 means that you are performing better than random guesses)


In [0]:
!rm -Rf a2_m2.parquet

In [0]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m2.json')

In [0]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2019-02-10 19:55:42--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2.2K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,289       --.-K/s   in 0s      

2019-02-10 19:55:43 (21.2 MB/s) - ‘rklib.py’ saved [2289/2289]



In [0]:
!zip -r a2_m2.json.zip a2_m2.json

  adding: a2_m2.json/ (stored 0%)
  adding: a2_m2.json/.part-00000-7c46e53a-8e60-4ca8-8cb8-08fe69b9092f.json.crc (stored 0%)
  adding: a2_m2.json/_SUCCESS (stored 0%)
  adding: a2_m2.json/part-00000-7c46e53a-8e60-4ca8-8cb8-08fe69b9092f.json (deflated 84%)
  adding: a2_m2.json/._SUCCESS.crc (stored 0%)


In [0]:
!base64 a2_m2.json.zip > a2_m2.json.zip.base64

In [0]:
from rklib import submit
key = "******"
part = "*****"
email = "annmargaret.tutu@icloud.com"
secret = "******"

with open('a2_m2.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"LTL4F","id":"f_F-qCtuEei_fRLwaVDk3g~LTL4F~LhBUQi2gEem1YxJomMervg","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
